In [2]:
import requests
import json
import pandas as pd
import numpy as np
import bs4
import glob
import os
import time
import re

In [3]:
response = requests.get('https://stardewvalleywiki.com/Stardew_Valley_Wiki').content

In [4]:
fish = bs4.BeautifulSoup(requests.get('https://stardewvalleywiki.com/Fish').content, features='lxml')

In [5]:
table = fish.find('tbody')
rows1 = table.find_all('tr')

In [6]:
def extract_fish(num, rows):
    fish_df = pd.DataFrame(columns=['Name', 'Location', 'Time', 'Season', 'Weather', 'Used In'])
    for i in range(num, len(rows), 13):
        row = rows[i]
        atts = row.find_all('td')
        name = atts[1].text.strip()
        location = atts[30].text.strip()
        time = atts[31].text.strip()
        season = atts[32].text.strip().split('\xa0')
        weather = atts[33].text.strip()
        if 37 >= len(atts):
            used_in = []
        else:
            spans = atts[37].find_all('span')
            ps = atts[37].find_all('p')
            used_in = []
            for i in spans:
                used_in.append(i.text.strip())
            for j in ps:
                used_in.append(j.text.strip().replace('\xa0', ' '))
        df_dict = {'Name': name, 'Location': location, 'Time': time, 'Season': season, 'Weather': weather, 'Used In': used_in}
        fish_df = fish_df.append(df_dict, ignore_index=True)
    return fish_df


first_fish = extract_fish(1, rows1)
rows2 = fish.find_all('table', attrs={'class': 'wikitable sortable'})[1].find_all('tr')

In [7]:
second_fish = pd.DataFrame(columns=['Name', 'Location', 'Time', 'Season', 'Weather', 'Used In'])
for i in range(1, len(rows2), 13):
    row = rows2[i]
    atts = row.find_all('td')
    name = atts[1].text.strip()
    location = 'Night Market Submarine'
    time = '5pm - 2am'
    season = 'Winter 15th - Winter 17th'
    weather = 'Any'
    used_in = []
    df_dict = {'Name': name, 'Location': location, 'Time': time, 'Season': season, 'Weather': weather, 'Used In': used_in}
    second_fish = second_fish.append(df_dict, ignore_index=True)
    
second_fish

,Name,Location,Time,Season,Weather,Used In
0,Midnight Squid,Night Market Submarine,5pm - 2am,Winter 15th - Winter 17th,Any,[]
1,Spook Fish,Night Market Submarine,5pm - 2am,Winter 15th - Winter 17th,Any,[]
2,Blobfish,Night Market Submarine,5pm - 2am,Winter 15th - Winter 17th,Any,[]


In [8]:
rows3 = fish.find_all('table', attrs={'class': 'wikitable sortable'})[2].find_all('tr')
third_fish = extract_fish(1, rows3)
third_fish

,Name,Location,Time,Season,Weather,Used In
0,Crimsonfish,East Pier on The Beach. Requires level 5 fishing.,Any,[Summer],Any,[]
1,Angler,North of JojaMart on the wooden plank bridge. ...,Any,[Fall],Any,[]
2,Legend,The Mountain Lake near the log. Requires level...,Any,[Spring],Rain,[]
3,Glacierfish,South end of Arrowhead Island in Cindersap For...,Any,[Winter],Any,[]
4,Mutant Carp,The Sewers,Any,[All Seasons],Any,[]


In [9]:
rows4 = fish.find_all('table', attrs={'class': 'wikitable sortable'})[4].find_all('tr')
atts = rows4[6].find_all('td')
atts[33].find_all('span')[0].text.strip()

'Chowder'

In [10]:
def extract_fish2(rows):
    fish_df = pd.DataFrame(columns=['Name', 'Location', 'Time', 'Season', 'Weather', 'Used In'])
    lr = len(rows)
    for i in range(lr):
        row = rows[i]
        atts = row.find_all('td')
        if len(atts) < 5:
            continue
        else:
            time = 'Any'
            season = 'All'
            weather = 'Any'
            used_in = []
            if len(atts) == 16:
                name = atts[1].text.strip()
                location = atts[12].text.strip()
                for i in atts[15].find_all('span'):
                    used_in.append(i.text.strip())
                for i in atts[15].find_all('p'):
                    used_in.append(i.text.strip().replace('\xa0', ' '))
            elif len(atts) == 34:
                name = atts[1].text.strip().strip('[3]')
                location = atts[30].text.strip()
                for i in atts[33].find_all('span'):
                    used_in.append(i.text.strip())
                for i in atts[33].find_all('p'):
                    used_in.append(i.text.strip().replace('\xa0', ' '))
                    
        dict_df = {'Name': name, 'Location': location, 'Time': time, 'Season': season, 'Weather': weather, 'Used In': used_in}
        fish_df = fish_df.append(dict_df, ignore_index=True)
    return fish_df

fourth_fish = extract_fish2(rows4)
fourth_fish

,Name,Location,Time,Season,Weather,Used In
0,Lobster,Ocean,Any,All,Any,"[Lobster Bisque, ""Wanted Lobster"" Quest, Ellio..."
1,Clam,Ocean,Any,All,Any,"[Chowder, Speed-Gro]"
2,Crayfish,Freshwater,Any,All,Any,[Fish Stew]
3,Crab,Ocean,Any,All,Any,"[Crab Cakes, Crab Cakes]"
4,Cockle,Ocean,Any,All,Any,[]
5,Mussel,Ocean,Any,All,Any,[Fish Stew]
6,Shrimp,Ocean,Any,All,Any,"[Tom Kha Soup, Shrimp Cocktail]"
7,Snail,Freshwater,Any,All,Any,"[Escargot, Vincent (loved Gift)]"
8,Periwinkle,Freshwater,Any,All,Any,"[Strange Bun, Fish Stew]"
9,Oyster,Ocean,Any,All,Any,[]


In [11]:
all_fish = pd.concat([first_fish, second_fish, third_fish, fourth_fish], ignore_index=True)
all_fish

,Name,Location,Time,Season,Weather,Used In
0,Pufferfish,"Ocean, Ginger Island oceans",12pm – 4pm,[Summer(All Seasons on Ginger Island)],Sun,"[Specialty Fish Bundle, ""Aquatic Research"" Que..."
1,Anchovy,Ocean,Anytime,"[Spring, Fall]",Any,[]
2,Tuna,"Ocean, Ginger Island oceans",6am – 7pm,"[Summer, Winter(All Seasons on Ginger Island)]",Any,"[Ocean Fish Bundle, Fish Taco]"
3,Sardine,Ocean,6am – 7pm,"[Spring, Fall, Winter]",Any,"[Ocean Fish Bundle, Dish o' The Sea]"
4,Bream,River (Town+Forest),6pm – 2am,[All Seasons],Any,"[Night Fishing Bundle, Baked Fish]"
...,...,...,...,...,...,...
60,Mussel,Ocean,Any,All,Any,[Fish Stew]
61,Shrimp,Ocean,Any,All,Any,"[Tom Kha Soup, Shrimp Cocktail]"
62,Snail,Freshwater,Any,All,Any,"[Escargot, Vincent (loved Gift)]"
63,Periwinkle,Freshwater,Any,All,Any,"[Strange Bun, Fish Stew]"
